In [ ]:
pip install gradio

In [5]:
import gradio as gr
from langchain_groq import ChatGroq

# Initialize the chatbot with a specific model and temperature
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,  # A higher temperature for more empathetic responses
    groq_api_key='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'
)

# Predefined responses for greetings and bot-related queries
greeting_responses = {
    "hello": "Hello! How can I help you today?",
    "hi": "Hi there! What would you like to talk about?",
    "hey": "Hey! I'm here to support you.",
    "good morning": "Good morning! How are you feeling today?",
    "good afternoon": "Good afternoon! How can I assist you?",
    "good evening": "Good evening! How's your day been?",
}

bot_related_responses = {
    "who are you": "I’m a mental health support chatbot. I'm here to provide empathetic responses and assist with questions related to the mind and emotions.",
    "what can you do": "I can help answer questions about mental health, thoughts, feelings, and other mind-related topics. Feel free to ask anything!",
    "tell me about yourself": "I’m designed to offer emotional support and help with mind-related topics, but I’m not a substitute for professional help. How can I assist you today?"
}

def is_mind_related_question(prompt):
    """
    Checks if the user's prompt is related to the mind, including various mental processes
    like thinking, visualizing, imagining, and others.
    """
    mind_related_keywords = [
        "mind", "brain", "thoughts", "thinking", "feelings", "emotions", "mental health", 
        "anxiety", "depression", "stress", "therapy", "counseling", "self-care", 
        "motivation", "focus", "concentration", "memory", "learning", "intelligence",
        "daydreaming", "mood", "meditation", "mindfulness", "behavior", "imagine",
        "visualize", "visualizing", "recall", "dream", "dreaming", "fantasize", "fantasy",
        "ruminate", "overthink", "self-talk", "inner dialogue", "mental imagery", 
        "cognition", "cognitive", "plan", "planning", "creative thinking", "subconscious"
    ]
    
    action_keywords = [
        "stop", "avoid", "control", "manage", "handle", "clear", "focus on", "concentrate on", 
        "remove", "stop thinking", "deal with", "visualize", "imagine", "plan", "process"
    ]

    # Convert the prompt to lowercase for easier comparison
    prompt_lower = prompt.lower()

    # Check for mind-related keywords directly
    for keyword in mind_related_keywords:
        if keyword in prompt_lower:
            return True

    # Allow prompts involving mental actions combined with activities
    for action in action_keywords:
        if action in prompt_lower and ("thinking" in prompt_lower or "mind" in prompt_lower or "imagine" in prompt_lower):
            return True

    return False

def handle_greetings_or_bot_questions(prompt):
    """
    Handles simple greetings and general queries about the bot.
    """
    prompt_lower = prompt.lower()
    
    # Check for greetings
    for greeting in greeting_responses:
        if greeting in prompt_lower:
            return greeting_responses[greeting]
    
    # Check for bot-related queries
    for query in bot_related_responses:
        if query in prompt_lower:
            return bot_related_responses[query]
    
    return None

# Chat function with conversation history handling
def chatbot_response(user_prompt, chat_history):
    """
    Handles the user's input and appends to the conversation history.
    The chatbot generates responses based on the entire conversation history.
    """
    # Check if it's a greeting or a bot-related question
    predefined_response = handle_greetings_or_bot_questions(user_prompt)
    
    if predefined_response:
        chat_history.append(("Bot", predefined_response))
        return chat_history

    # Append the user's query to the chat history
    chat_history.append(("User", user_prompt))
    
    # Check if the user's prompt is related to the mind
    if not is_mind_related_question(user_prompt):
        chat_history.append(("Bot", "Please ask only about questions related to the mind (e.g., thoughts, feelings, emotions, imagination)."))
        return chat_history
    
    # Prepare the conversation as a string, appending each previous turn
    conversation = "\n".join([f"{speaker}: {message}" for speaker, message in chat_history])

    # Prepend a guiding prompt to help the model respond empathetically
    mind_related_prompt = f"I’m here to listen and offer support. Respond with empathy and understanding. Here is the conversation so far:\n{conversation}\n{user_prompt}"

    try:
        # Get the model's response
        res = llm.invoke(mind_related_prompt)
        # Append the model's response to the chat history
        chat_history.append(("Bot", res.content))
    except Exception as e:
        chat_history.append(("Bot", "Sorry, there was an issue processing your request. Please try again."))
    
    return chat_history

# Gradio interface
def launch_gradio_interface():
    # Create a Gradio interface with a chat-like textbox and output
    with gr.Blocks() as demo:
        # Create a state to hold the chat history
        chat_history = gr.State([])

        # Chat interface: input textbox and output box
        with gr.Row():
            user_input = gr.Textbox(label="Ask your mind-related question")
            chat_output = gr.Chatbot(label="Mental Health Support Chatbot")

        # Button to submit the user query
        submit_button = gr.Button("Submit")

        # Function to update the chat history with each new input
        def update_chat(user_prompt, chat_history):
            chat_history = chatbot_response(user_prompt, chat_history)
            return chat_history, chat_history, ""

        # Button to clear only the user prompt
        def clear_prompt():
            return ""

        # Button to clear both the user prompt and the chat history
        def clear_chat():
            return [], [], ""

        # Add a button to clear the user input only
        clear_prompt_button = gr.Button("Clear Prompt")

        # Add a button to clear both the user input and the chat history
        clear_chat_button = gr.Button("Clear Chat")

        # Link the user input and chat history updates
        submit_button.click(update_chat, inputs=[user_input, chat_history], outputs=[chat_output, chat_history, user_input])

        # Link the clear buttons to their respective functions
        clear_prompt_button.click(clear_prompt, inputs=[], outputs=[user_input])
        clear_chat_button.click(clear_chat, inputs=[], outputs=[chat_output, chat_history, user_input])

    # Launch the Gradio UI
    demo.launch()

# Run the Gradio interface
launch_gradio_interface()


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
